In [1]:
import torch
import random
import numpy as np
import regex
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def split_train_val_test(df, props=[.8, .1, .1]):
    assert round(sum(props), 2) == 1 and len(props) >= 2
    train_df, test_df, val_df = None, None, None

    train_size = int(props[0] * len(df))
    val_size =  train_size + int(props[1] * len(df))
    test_size =val_size + int(props[2] * len(df)) 
    train_df = df.iloc[0:train_size]
    val_df = df.iloc[train_size:val_size]
    test_df = df.iloc[val_size:test_size]
    
    return train_df, val_df, test_df

In [3]:
import gensim.downloader as api

def download_word2vec_embeddings():
    print("Downloading pre-trained word embeddings from: word2vec-google-news-300.\n" 
          + "Note: This can take a few minutes.\n")
    wv = api.load("word2vec-google-news-300")
    print("\nLoading complete!\n" +
          "Vocabulary size: {}".format(len(wv.vocab)))
    return wv


In [4]:
# Opening and preprocessing input file
import gensim.models
import pandas as pd
import nltk
from tqdm import tqdm
from src.preprocess import clean_text

data = pd.read_csv('train.csv', quotechar='"')
data.sample(frac=1)


# to convert authors into numbers
author_to_number = {
    'EAP': 0,
    'HPL': 1,
    'MWS': 2
    
}

# lowercase, removing punctuation and tookenize sentences. Converting labels to int
training_text = ""
for i in range(len(data)):

    data['text'][i] = nltk.word_tokenize(regex.sub(r'[^\w\s]', '',data['text'][i].lower()))
    data['author'][i] = author_to_number[data['author'][i]]

print(data[0:10])

from src.dataset import *

# Splitting dataset and generating vocab
train_df, val_df, test_df = split_train_val_test(data)
train_vocab, reversed_vocab = generate_vocab_map(train_df)



        id                                               text author
0  id26305  [this, process, however, afforded, me, no, mea...      0
1  id17569  [it, never, once, occurred, to, me, that, the,...      1
2  id11008  [in, his, left, hand, was, a, gold, snuff, box...      0
3  id27763  [how, lovely, is, spring, as, we, looked, from...      2
4  id12958  [finding, nothing, else, not, even, gold, the,...      1
5  id22965  [a, youth, passed, in, solitude, my, best, yea...      2
6  id09674  [the, astronomer, perhaps, at, this, point, to...      0
7  id13515  [the, surcingle, hung, in, ribands, from, my, ...      0
8  id19322  [i, knew, that, you, could, not, say, to, your...      0
9  id00912  [i, confess, that, neither, the, structure, of...      2


In [5]:
DOWNLOAD = True
# Use fastext or word2vec
FASTTEXT = False
WINDOW_SIZE = 5

EMBEDDING_DIM = 300
HIDDEN_DIM = 128
NUM_LAYERS = 1
BIDIRECTIONAL = True


In [6]:
# Downloading or generating word2vec embeddings

if DOWNLOAD:
    model = download_word2vec_embeddings()
else:
    if not FASTTEXT:
        model = gensim.models.Word2Vec(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
    else:
        model = gensim.models.FastText(sentences=train_df['text'], size=EMBEDDING_DIM, window=WINDOW_SIZE)
                        

Note: This can take a few minutes.


Loading complete!
Vocabulary size: 3000000


In [7]:
from src.dataset import HeadlineDataset
from torch.utils.data import RandomSampler

train_dataset = HeadlineDataset(train_vocab, train_df,model.wv, FASTTEXT)
val_dataset = HeadlineDataset(train_vocab, val_df,model.wv, FASTTEXT)
test_dataset = HeadlineDataset(train_vocab, test_df,model.wv, FASTTEXT)

# Now that we're wrapping our dataframes in PyTorch datsets, we can make use of PyTorch Random Samplers.
train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

/tmp/ipykernel_24500/1539958925.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  train_dataset = HeadlineDataset(train_vocab, train_df,model.wv, FASTTEXT)
/tmp/ipykernel_24500/1539958925.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  val_dataset = HeadlineDataset(train_vocab, val_df,model.wv, FASTTEXT)
/tmp/ipykernel_24500/1539958925.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  test_dataset = HeadlineDataset(train_vocab, test_df,model.wv, FASTTEXT)


In [8]:
from torch.utils.data import DataLoader
from src.dataset import collate_fn
BATCH_SIZE = 16
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

for x, y in test_iterator:
    print(x,y)
    break

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1797, -0.0913, -0.1553,  ..., -0.1143, -0.0378, -0.1514],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.2256, -0.0195,  0.0908,  ...,  0.0282, -0.1777, -0.0060],
         [ 0.0771, -0.1396,  0.1445,  ..., -0.0845,  0.2002, -0.3145],
         [-0.0332, -0.0327, -0.0598,  ..., -0.0058,  0.1299, -0.0209],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0884, -0.0317, -0.1226,  ...,  0.0234,  0.2480, -0.1177],
         [-0.0581,  0.0581,  0.0133,  ..., -0

### Modeling

In [9]:
from src.models import ClassificationModel

model = ClassificationModel(len(train_vocab),embedding_dim=EMBEDDING_DIM,hidden_dim = HIDDEN_DIM,num_layers = NUM_LAYERS,bidirectional = BIDIRECTIONAL)


In the following cell, **instantiate the model with some hyperparameters, and select an appropriate loss function and optimizer.** 

Hint: we already use sigmoid in our model. What loss functions are availible for binary classification? Feel free to look at PyTorch docs for help!

In [10]:
from torch.optim import AdamW

criterion, optimizer = torch.nn.CrossEntropyLoss(), torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

### Part 3: Training and Evaluation [10 Points]
The final part of this HW involves training the model, and evaluating it at each epoch. **Fill out the train and test loops below.**

In [11]:
# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(iterator):
        optimizer.zero_grad()

        prediction = model(x)
        prediction = torch.squeeze(prediction)
        # y = y.round()
        # y = y.long()

 
        loss = criterion(prediction,y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    return total_loss

# returns:
# - true: a Python boolean array of all the ground truth values 
#         taken from the dataset iterator
# - pred: a Python boolean array of all model predictions. 
def val_loop(model, criterion, iterator):
    true, pred = [], []
    for x, y in tqdm(iterator):
    
        preds = model(x)
        preds = torch.squeeze(preds)
        for i_batch in range(len(y)):
            true.append(y[i_batch])
            pred.append(torch.argmax(preds[i_batch]))
            
    return true, pred


In [12]:
# To test your eval implementation, let's see how well the untrained model does on our dev dataset.
# It should do pretty poorly.
from src.eval_utils import binary_macro_f1, accuracy
true, pred = val_loop(model, criterion, val_iterator)
# print(binary_macro_f1(true, pred))
# print(accuracy(true, pred))


100%|██████████| 123/123 [00:04<00:00, 27.71it/s]


ZeroDivisionError: division by zero

### Actually training the model

In [ ]:
TOTAL_EPOCHS = 7
for epoch in range(TOTAL_EPOCHS):
    train_loss = train_loop(model, criterion, train_iterator)
    true, pred = val_loop(model, criterion, val_iterator)
    print(f"EPOCH: {epoch}")
    print(f"TRAIN LOSS: {train_loss}")
    print(f"VAL F-1: {binary_macro_f1(true, pred)}")
    print(f"VAL ACC: {accuracy(true, pred)}")


100%|██████████| 123/123 [00:04<00:00, 25.73it/s]


EPOCH: 0
TRAIN LOSS: 1067.5374600887299
VAL F-1: 0.4037390884816184
VAL ACC: 0.40725600408788964


100%|██████████| 123/123 [00:05<00:00, 20.85it/s]


EPOCH: 1
TRAIN LOSS: 1054.2395766973495
VAL F-1: 0.43301331334026644
VAL ACC: 0.45426673479816043


100%|██████████| 123/123 [00:04<00:00, 26.26it/s]


...

100%|██████████| 123/123 [00:04<00:00, 28.39it/s]


EPOCH: 4
TRAIN LOSS: 1045.43911921978
VAL F-1: 0.48630174932921094
VAL ACC: 0.48696985181400104


100%|██████████| 123/123 [00:05<00:00, 23.67it/s]


EPOCH: 5
TRAIN LOSS: 1046.2310537695885
VAL F-1: 0.4753077765806211
VAL ACC: 0.4803270311701584


 26%|██▌       | 256/979 [00:34<02:23,  5.03it/s]

We can also look at the models performance on the held-out test set, using the same val_loop we wrote earlier.

In [ ]:
true, pred = val_loop(model, criterion, test_iterator)
print(f"TEST F-1: {binary_macro_f1(true, pred)}")
print(f"TEST ACC: {accuracy(true, pred)}")

100%|██████████| 123/123 [00:04<00:00, 25.34it/s]


TEST F-1: 0.5651973994448763
TEST ACC: 0.5891670924885029
